In [19]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np

import pandas as pd
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import time
import json
from preprocessing_dailydialogue import *
import pickle

In [318]:
# Define constant
# Default word tokens
#
torch.autograd.set_detect_anomaly(True)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
MAX_LENGTH = 10  # Maximum sentence length to consider
MIN_COUNT = 3    # Minimum word count threshold for trimming
save_dir = os.path.join("data", "save")
emo_dict = { 0: 'neutral', 1: 'joy', 2: 'anger', 
            3: 'sadness',4:'fear'}
emo2idx = {value:key for key,value in emo_dict.items()}


In [319]:
USE_CUDA

True

# get data from pickle (No preprocessing required)

In [320]:
with open('processed_train.pickle','rb') as f:
    pairs = pickle.load(f)
    pairs_emotion = pickle.load(f)
    voc = pickle.load(f)
with open('processed_test.pickle','rb') as f:
    pairs_t = pickle.load(f)
    pairs_emotion_t = pickle.load(f)


In [323]:
test_batch = batch2TrainData(voc,list(range(1000)),pairs[-2000:],pairs_emotion[-2000:])
pairs = pairs[:-2000]
pairs_emotion = pairs_emotion[:-2000]
test_pairs = pairs[-2000:]
test_pairs_emotion = pairs_emotion[-2000:]


In [324]:
test_batch[-2].sum()

tensor(6386)

In [325]:
len(pairs),len(pairs_emotion)



(64044, 64044)

# Convert data to tensor

In [326]:
# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(list(range(len(pairs)))) for _ in range(small_batch_size)],pairs,pairs_emotion)
input_variable,input_emotion, lengths, target_variable,target_emotion, mask, max_target_len = batches

print("input_variable:", input_variable)
print('Input_emotion:',input_emotion)
print("lengths:", lengths)
print("target_variable:", target_variable)
print('target_emotion:',target_emotion)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  22,  680,   91, 4422,   11],
        [ 181,  141,    8,    6,   36],
        [1931,    8,  182,   65,    8],
        [  18, 1271,  163,   15,  251],
        [1931,  691,   10,   16,   13],
        [2110,  304,  254, 7029,    2],
        [  35, 1272,    6,    6,    0],
        [2662,   56,    2,    2,    0],
        [   6,    2,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
Input_emotion: tensor([0, 0, 2, 4, 4])
lengths: tensor([10,  9,  8,  8,  6])
target_variable: tensor([[   8,   77,  668, 1674,  594],
        [  21,   16,   59,   37,   59],
        [ 509,  428, 3216, 4735, 1931],
        [ 413,    6,  582,   15,   91],
        [2782,  141,   91,  152,    8],
        [   6,    8,   97,   65,   13],
        [   2,   64, 5399,  760,    2],
        [   0,    6,   56,  850,    0],
        [   0,    2,    2,    6,    0],
        [   0,    0,    0,    2,    0]])
target_emotion: tensor([0, 3, 2, 3, 4])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1,

# Encoder + Attention

In [327]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden
# Luong attention layer
'''
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)   
        '''
print('Implement attention inside ECM')

Implement attention inside ECM


# ECM: Internal memory + Attention

In [328]:
class ECMWrapper(nn.Module):
    '''
    Internal memory module
    '''
    def __init__(self,hidden_size,state_size,emo_size,num_emotion,embedding,emotion_embedding,gru,device):
        '''
        hidden_size: hidden input dimension
        state_size: state vector size (input a word so hidden size)
        emo_size: emotional embedding size (usually similar to hidden_size)
        num_emotion: number of emotion categories
        '''
        super(ECMWrapper,self).__init__()
        self.hidden_size = hidden_size
        self.state_size = state_size
        self.emo_size = emo_size
        self.num_emotion = num_emotion
        self.device = device
        # read gate dimensions (word_embedding + hidden_input + context_input)
        self.read_g = nn.Linear(self.hidden_size + self.hidden_size + self.hidden_size,self.emo_size)
        # write gate
        self.write_g = nn.Linear(self.state_size, self.emo_size)
        # GRU output input dimensions = state_last + context + emotion emb + internal memory
        self.gru = gru
        self.emotion_embedding = emotion_embedding
        self.embedding = embedding
        # attention layer
        self.attn1 = nn.Linear(self.hidden_size,self.hidden_size)
        self.attn2 = nn.Linear(self.hidden_size,self.hidden_size)
        self.concat = nn.Linear(self.hidden_size, 1)
    def forward(self,word_input,decoder_output,emotion_input,context_input,last_hidden,memory):
        '''
        Last hidden == prev_cell_state
        last word embedding = word_input
        last hidden input = h
        last_rnn_output = logits before softmax
        memory = encoder_outputs
        '''
        # get embedding of input word and emotion
        if decoder_output is None:
            decoder_output = torch.zeros(word_input.shape[1],self.hidden_size,dtype=torch.float,device = self.device)
            decoder_output = decoder_output.unsqueeze(0)
            context_input = self._compute_context(decoder_output,memory)
        last_word_embedding = self.embedding(word_input)
        read_inputs = torch.cat((last_word_embedding,decoder_output,context_input), dim = -1)
        # compute read input
        # write to emotion embedding
        emotion_input = self._read_internal_memory(read_inputs,emotion_input)
        # pass everything to GRU
        # decoder_output: logits from last rnn unit
        X = torch.cat([last_word_embedding,decoder_output, context_input, emotion_input], dim = -1)
        rnn_output, hidden = self.gru(X,last_hidden)
        # write input
        # update states
        # write to emotion embedding
        new_M_emo = self._write_internal_memory(emotion_input,rnn_output) # new emotion_input
        new_context = self._compute_context(rnn_output,memory)
        return rnn_output, hidden, new_M_emo, new_context
    def _compute_context(self,rnn_output,memory):
        '''
        Compute context
        '''
        rnn_output = rnn_output.unsqueeze(dim=-2).squeeze(0) # make shape (batch,1,hidden_size)
        memory = memory.permute(1,0,2)
        Wq = self.attn1(rnn_output)
        Wm = self.attn2(memory)
        concat = (Wq + Wm).tanh()
        e = self.concat(concat).squeeze(2)
        attn_score = torch.softmax(e,dim = 1).unsqueeze(1)
        context = torch.bmm(attn_score,memory).squeeze(1)
        return context.unsqueeze(0)
    def _read_internal_memory(self,read_inputs,emotion_input):
        """
        Read the internal memory
            emotion_input: [batch_size, emo_hidden_size]
            read_inputs: [batch_size, d] d= [last_word_embedding;decoder_output;context_input]
        Returns:
            output: [batch_size, emo__hidden_size]
        """
        read_inputs = self.read_g(read_inputs)
        M_read = torch.sigmoid(read_inputs)
        return emotion_input * M_read
    def _write_internal_memory(self,emotion_input,rnn_output):
        """
        Write the internal memory
            emotion_input: [batch_size, emo_hidden_size]
            rnn_output: [batch_size, hidden_size]
        Returns:
            output: [batch_size, emo_hidden_size]
        """
        M_write = torch.sigmoid(self.write_g(rnn_output))
        return emotion_input * M_write
    
    

# Decoder part

In [329]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding,emotion_embedding, hidden_size, output_size,device,ememory=None, n_layers=1, dropout=0.1,num_emotions = 7,batch_size = 64):
        super(LuongAttnDecoderRNN, self).__init__()
        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.batch_size = batch_size
        self.dropout = dropout
        self.num_emotions = num_emotions
        self.device = device
        # Define layers
        self.embedding = embedding
        # define emotion embedding
        self.emotion_embedding = emotion_embedding
        self.embedding_dropout = nn.Dropout(dropout)
        #self.emotion_embedding_dropout = nn.Dropout(dropout)
        # dimension
        self.gru = nn.GRU(hidden_size + hidden_size + hidden_size + hidden_size , hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        # using in Luong et al. attention mechanism.
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.g_out = nn.Linear(hidden_size, output_size)
        self.e_out = nn.Linear(hidden_size, output_size)
        #self.attn = Attn(attn_model, hidden_size)
        self.internal_memory = ECMWrapper(hidden_size,hidden_size,
                                          hidden_size,self.num_emotions,
                                          self.embedding,self.emotion_embedding,self.gru,device)
        # read external from outside
        self.external_memory = ememory
        # emotional output linear layer 
        self.emotion_word_output_layer = nn.Linear(self.hidden_size,output_size)
        # emotional gate/ choice layer
        self.alpha_layer = nn.Linear(hidden_size,1)
        
    def forward(self, input_step,input_step_emotion, last_hidden
                ,input_context, encoder_outputs,last_rnn_output = None):
        '''
        
        '''
        if not torch.is_floating_point(input_step_emotion):
            input_step_emotion = self.emotion_embedding(input_step_emotion)
        rnn_output, hidden, new_M_emo,context = self.internal_memory(input_step,last_rnn_output,input_step_emotion,
                                                            input_context,last_hidden,encoder_outputs)
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        concat_input = torch.cat((rnn_output, context), -1)
        concat_output = torch.tanh(self.concat(concat_input))
        # this part is not using inside ECM (?)
        if self.external_memory is not None:
            # generic logits
            output_g = self.g_out(concat_output)
            # emotional logits
            output_e = self.e_out(concat_output)
            # external memory gate - alpha
            g = torch.sigmoid(self.alpha_layer(concat_output))
            # compute softmax output
            output_e = F.softmax(output_e,dim=1) * (g)
            output_g = F.softmax(output_g,dim=1) * (1 - g)
            #output = torch.cat((output_e,output_g),dim=1)
            #idx = torch.cat((idx_e,idx_g),dim = 0)
            #idx_sort,_ = torch.sort(idx,dim = 0,descending = False)
            #output = output[:,idx_sort]
            output_g = output_g.squeeze(0)
            output_e = output_e.squeeze(0)
            output = output_g + output_e
        else:
            # Predict next word using Luong eq. 6
            output = self.out(concat_output)
            # generic output
            output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden, new_M_emo, context,concat_output,g

# NLL Loss + Internal Memory Loss

In [330]:
def maskNLLLoss_IMemory(inp, target, mask,M_emo,external_memory,alpha):
    '''
    When external memory input will be a tuple with 4 elements
    '''
    nTotal = mask.sum()
    # find 1,0
    qt = torch.gather(external_memory.view(-1,1),0,target.view(-1,1)).type_as(M_emo)
    # cross entropy loss
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    # internal emotional loss
    eos_mask = (target == 2) # 2 is EOS token
    eos_mask = eos_mask.type_as(M_emo)
    internal_memory_loss = torch.norm(M_emo,dim = 2) * eos_mask
    internal_memory_loss = internal_memory_loss.squeeze(0)
    # external
    external_memory_loss = (- qt * torch.log(alpha)).reshape(-1) 
    loss = crossEntropy.masked_select(mask).sum() + \
    internal_memory_loss.masked_select(mask).sum() + \
    external_memory_loss.masked_select(mask).sum()
    loss = loss.to(device)
    return loss, nTotal.item()



# Single Train

In [331]:
def compute_perplexity(loss):
    return np.exp(loss)
def train(input_variable, lengths, target_variable,target_variable_emotion,
          mask, max_target_len, encoder, decoder, embedding,emotion_embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):
    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # num_samples in this batch
    num_samples = input_variable.shape[1]
    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    target_variable_emotion = target_variable_emotion.to(device)
    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(num_samples)]])
    decoder_input = decoder_input.to(device)
    
    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    
    # Set initial context value,last_rnn_output, internal_memory
    context_input = torch.zeros(num_samples,hidden_size,dtype=torch.float,device=device) #torch.FloatTensor(batch_size,hidden_size)
    # Determine if we are using teacher forcing this iteration
    if random.random() < teacher_forcing_ratio:
        use_teacher_forcing = True  
    else:
        use_teacher_forcing = False
    use_teacher_forcing = False
    # initialize value for rnn_output
    rnn_output = None
    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden,target_variable_emotion,context_input,rnn_output,g = decoder(
                decoder_input,target_variable_emotion, decoder_hidden,
                context_input, encoder_outputs,rnn_output
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss_IMemory(decoder_output, target_variable[t], mask[t],target_variable_emotion,decoder.external_memory,g)
            loss += mask_loss
            print_losses.append(mask_loss.item()) # print average loss
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden,target_variable_emotion,context_input,rnn_output,g = decoder(
                decoder_input,target_variable_emotion, decoder_hidden,
                context_input,encoder_outputs,rnn_output
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(num_samples)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss_IMemory(decoder_output, target_variable[t], mask[t],target_variable_emotion,decoder.external_memory,g)
            loss += mask_loss
            print_losses.append(mask_loss.item()) # print average loss
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals,loss
def evaluate_performance(input_variable, lengths, target_variable,target_variable_emotion,
          mask, max_target_len, encoder, decoder):
    # test mode
    
    encoder.eval()
    decoder.eval()
    # num_samples in this batch
    num_samples = input_variable.shape[1]
    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    target_variable_emotion = target_variable_emotion.to(device)
    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(num_samples)]])
    decoder_input = decoder_input.to(device)
    
    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    # Set initial context value,last_rnn_output, internal_memory
    context_input = torch.zeros(num_samples,hidden_size,dtype=torch.float,device=device) #torch.FloatTensor(batch_size,hidden_size)
    # initial value for rnn output
    rnn_output = None
    # forward pass to generate all sentences
    for t in range(max_target_len):
        decoder_output, decoder_hidden,target_variable_emotion,context_input,rnn_output,g = decoder(
            decoder_input,target_variable_emotion, decoder_hidden,
            context_input,encoder_outputs,rnn_output
        )
        # No teacher forcing: next input is decoder's own current output
        #output_g,output_e = decoder_output
        #output_g = output_g.squeeze(0)
        #output_e = output_e.squeeze(0)
        #output = output_g + output_e
        _, topi = decoder_output.topk(1)
        decoder_input = torch.LongTensor([[topi[i][0] for i in range(num_samples)]])
        decoder_input = decoder_input.to(device)
        # Calculate and accumulate loss
        mask_loss, nTotal = maskNLLLoss_IMemory(decoder_output, target_variable[t], mask[t],target_variable_emotion,decoder.external_memory,g)
        loss += mask_loss
        print_losses.append(mask_loss.item()) # print average loss
        n_totals += nTotal
    # back to train mode
    encoder.train()
    decoder.train()
    return sum(print_losses) / n_totals

# Train Iteration

In [339]:
def trainIters(model_name, voc, pairs,pairs_emotion, 
               encoder, decoder, encoder_optimizer,
               decoder_optimizer, embedding,emotion_embedding, 
               encoder_n_layers, decoder_n_layers, save_dir, 
               n_iteration, batch_size, print_every, save_every, 
               clip,corpus_name,external_memory,test_pairs,test_pairs_emotion):
    loadFilename=None
    # Load batches for each iteration
    #training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      #for _ in range(n_iteration)]
    print('Loading Training data ...')
    length_pairs = len(pairs)
    #training_batches = [batch2TrainData(voc, [random.choice(range(length_pairs)) for _ in range(batch_size)],
    #                                   pairs,pairs_emotion) for _ in range(n_iteration)]
    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = batch2TrainData(voc, [random.choice(range(length_pairs)) for _ in range(batch_size)],
                                       pairs,pairs_emotion)
        # to save the data that causes error
        #with open('wrong_data.pickle','rb') as f:
        #    training_batch = pickle.load(f)
        
        # Extract fields from batch
        input_variable,input_variable_emotion, lengths, target_variable,target_variable_emotion, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss,loss_tensor = train(input_variable, lengths, target_variable,target_variable_emotion,
                     mask, max_target_len, encoder,
                     decoder, embedding,emotion_embedding,
                     encoder_optimizer, decoder_optimizer, 
                     batch_size, clip)
        
        #print(loss_tensor)
        #if torch.isinf(loss_tensor) or torch.isnan(loss_tensor):
        #    with open('wrong_data.pickle','wb') as f:
        #        pickle.dump(training_batch,f)
        #        raise ValueError('NaN Found')
        print_loss += loss

        # Print progress
        if iteration % print_every == 0 or iteration == 1:
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            if iteration == 1:
                print_loss_avg = print_loss / 1
            else:
                print_loss_avg = print_loss / print_every
            perplexity = compute_perplexity(print_loss_avg)
            output1 = "Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}; Perplexity: {:.2f}".format(iteration, iteration / n_iteration * 100, print_loss_avg,perplexity)
            print(output1)
            test_length_pairs = len(test_pairs) 
            test_batch = batch2TrainData(voc, [random.choice(range(test_length_pairs)) for _ in range(batch_size)],
                                       test_pairs,test_pairs_emotion)
            input_variable,input_emotion, lengths, target_variable,target_emotion, mask, max_target_len = test_batch
            test_loss = evaluate_performance(input_variable,lengths, target_variable,target_emotion,mask,max_target_len,encoder,decoder)
            output2 = 'Loss on validation set {:.4f}; Perplexity:{:.2f}'.format(test_loss,compute_perplexity(test_loss))
            print(output2)
            with open(os.path.join(directory,'log.txt'),'a+') as f:
                f.write(output1 + '\n')
                f.write(output2 + '\n')
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict(),
                'external_memory':external_memory
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))
            
            

# Greedy Search

In [340]:
def print_param(model):
    for name,param in model.named_parameters():
        print(param)
        print(name,param.grad)

In [341]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder,num_word = None):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq,target_emotions,input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Set initial context value,last_rnn_output, internal_memory
        context_input = torch.zeros((1,hidden_size),dtype=torch.float,device=self.decoder.device)
        context_input = context_input.to(device)
        rnn_output = None
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden,target_emotions,context_input,rnn_output,g = decoder(
                decoder_input,target_emotions, decoder_hidden,
                context_input, encoder_outputs,rnn_output
            )
            # Obtain most likely word token and its softmax score
            output_g,output_e = decoder_output
            output_g = output_g.squeeze(0)
            output_e = output_e.squeeze(0)
            decoder_output = output_g + output_e
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

# Beam Search

In [342]:
class BeamSearchDecoder(nn.Module):
    def __init__(self, encoder, decoder,num_word):
        super(BeamSearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.num_word = num_word

    def forward(self, input_seq,target_emotions,input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_words_order = torch.zeros((1,self.num_word),device=decoder.device,dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        all_scores_array = torch.zeros((1,self.num_word),device=decoder.device,dtype=torch.float)
        # Set initial context value,last_rnn_output, internal_memory
        context_input = torch.zeros(1,hidden_size,dtype=torch.float)
        context_input = context_input.to(decoder.device)
        rnn_output = None
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden,target_emotions,context_input,rnn_output,g = decoder(
                decoder_input,target_emotions, decoder_hidden,
                context_input, encoder_outputs,rnn_output
            )
            # Obtain most likely word token and its softmax score
            output_g,output_e = decoder_output
            output_g = output_g.squeeze(0)
            output_e = output_e.squeeze(0)
            decoder_output = output_g + output_e
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            decoder_input_order = torch.argsort(decoder_output,dim=1,descending=True)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            all_scores_array = torch.cat((all_scores_array,decoder_output),dim = 0)
            all_words_order = torch.cat((all_words_order,decoder_input_order), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        print(all_scores_array)
        sequences = self.beam_search(all_scores_array,3)
        return sequences
    def beam_search(self,array,k):
        array = array.tolist()
        sequences = [[list(), 1.0]]
        # walk over each step in sequence
        for row in array:
            all_candidates = list()
            # expand each current candidate
            for i in range(len(sequences)):
                seq, score = sequences[i]
                for j in range(len(row)):
                    candidate = [seq + [j], score - np.log(row[j] + 1e-8)]
                    all_candidates.append(candidate)
            # order all candidates by score
            ordered = sorted(all_candidates, key=lambda tup:tup[1])
            # select k best
            sequences = ordered[:k]
        return sequences

# Build Model

In [ ]:
# Configure models
model_name = 'emotion_model'
corpus_name = 'ECM10_words'
attn_model = 'concat'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.2
batch_size = 128
# number of emotion
num_emotions = 5
# load external memory based vocab.
emotion_words = get_ememory('ememory.txt',voc)
# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None#'data/save/emotion_model/ECM10_words/2-2_500/10000_checkpoint.tar'
checkpoint_iter = 120
training = True
if loadFilename:
    training = False
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']
    emotion_words = checkpoint['external_memory']
    


print('Building encoder and decoder ...')
# Initialize word embeddings
emotion_words = emotion_words.to(device)
embedding = nn.Embedding(voc.num_words, hidden_size)
emotion_embedding = nn.Embedding(num_emotions, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding,emotion_embedding, hidden_size, 
                              voc.num_words,device, emotion_words,decoder_n_layers, dropout,num_emotions=num_emotions,batch_size = batch_size)
print(decoder.external_memory.device)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
    
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Emotion word counts: 3962
Building encoder and decoder ...
cuda:0
Models built and ready to go!


# Run training

In [ ]:
# Configure training/optimization
clip = 50
teacher_forcing_ratio = 0.1
learning_rate = 0.001
decoder_learning_ratio = 5.0
n_iteration = 2000000
print_every = 20
save_every = 5000


# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations

if training:
    print("Starting Training!")
    trainIters(model_name, voc, pairs,pairs_emotion, encoder, decoder, encoder_optimizer, decoder_optimizer,
               embedding,emotion_embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
               print_every, save_every, clip,corpus_name,emotion_words,test_pairs,test_pairs_emotion)
    
    

Building optimizers ...
Starting Training!
Loading Training data ...
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 5.2942; Perplexity: 199.19
Loss on validation set 4.8586; Perplexity:128.84
Iteration: 20; Percent complete: 0.0%; Average loss: 4.5976; Perplexity: 99.25
Loss on validation set 4.8474; Perplexity:127.41
Iteration: 40; Percent complete: 0.0%; Average loss: 4.8317; Perplexity: 125.43
Loss on validation set 4.8298; Perplexity:125.19
Iteration: 60; Percent complete: 0.0%; Average loss: 4.8336; Perplexity: 125.66
Loss on validation set 4.8247; Perplexity:124.55
Iteration: 80; Percent complete: 0.0%; Average loss: 4.8254; Perplexity: 124.64
Loss on validation set 4.8310; Perplexity:125.33
Iteration: 100; Percent complete: 0.0%; Average loss: 4.8227; Perplexity: 124.30
Loss on validation set 4.8334; Perplexity:125.64
Iteration: 120; Percent complete: 0.0%; Average loss: 4.8173; Perplexity: 123.63
Loss on validation set 4.7681; Perplexity:117.69

Loss on validation set 4.7020; Perplexity:110.17
Iteration: 1260; Percent complete: 0.1%; Average loss: 4.7162; Perplexity: 111.74
Loss on validation set 4.7431; Perplexity:114.79
Iteration: 1280; Percent complete: 0.1%; Average loss: 4.7118; Perplexity: 111.26
Loss on validation set 4.6605; Perplexity:105.69
Iteration: 1300; Percent complete: 0.1%; Average loss: 4.7081; Perplexity: 110.85
Loss on validation set 4.7018; Perplexity:110.14
Iteration: 1320; Percent complete: 0.1%; Average loss: 4.7031; Perplexity: 110.29
Loss on validation set 4.7735; Perplexity:118.34
Iteration: 1340; Percent complete: 0.1%; Average loss: 4.7107; Perplexity: 111.13
Loss on validation set 4.6985; Perplexity:109.78
Iteration: 1360; Percent complete: 0.1%; Average loss: 4.7051; Perplexity: 110.51
Loss on validation set 4.7476; Perplexity:115.30
Iteration: 1380; Percent complete: 0.1%; Average loss: 4.6932; Perplexity: 109.20
Loss on validation set 4.7246; Perplexity:112.69
Iteration: 1400; Percent complete:

Loss on validation set 4.6619; Perplexity:105.84
Iteration: 2520; Percent complete: 0.1%; Average loss: 4.6226; Perplexity: 101.76
Loss on validation set 4.5322; Perplexity:92.96
Iteration: 2540; Percent complete: 0.1%; Average loss: 4.6323; Perplexity: 102.74
Loss on validation set 4.6256; Perplexity:102.07
Iteration: 2560; Percent complete: 0.1%; Average loss: 4.6115; Perplexity: 100.64
Loss on validation set 4.6867; Perplexity:108.49
Iteration: 2580; Percent complete: 0.1%; Average loss: 4.6107; Perplexity: 100.55
Loss on validation set 4.6597; Perplexity:105.60
Iteration: 2600; Percent complete: 0.1%; Average loss: 4.6149; Perplexity: 100.98
Loss on validation set 4.6597; Perplexity:105.61
Iteration: 2620; Percent complete: 0.1%; Average loss: 4.6257; Perplexity: 102.07
Loss on validation set 4.6102; Perplexity:100.51
Iteration: 2640; Percent complete: 0.1%; Average loss: 4.6147; Perplexity: 100.95
Loss on validation set 4.6767; Perplexity:107.41


# Evaluation

In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence, emotions,max_length=MAX_LENGTH,beam_search = False):
    emotions = int(emotions)
    emotions = torch.LongTensor([emotions])
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    emotions = emotions.to(device)

    # indexes -> words
    if beam_search:
        sequences = searcher(input_batch, emotions, lengths, max_length)
        decoded_words = beam_decode(sequences,voc)
    else:
        # Decode sentence with searcher
        tokens, scores = searcher(input_batch, emotions, lengths, max_length)
        decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words

def beam_decode(sequences,voc):
    for each in sequences:
        for idxs in each:
            return [voc.index2word[idx] for idx in idxs[:-1]]
    
def evaluateInput(encoder, decoder, searcher, voc,emotion_dict,beam_search):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            for emotion in range(len(emotion_dict)):
                # Check if it is quit case
                if input_sentence == 'q' or input_sentence == 'quit': break
                # Normalize sentence
                input_sentence = normalizeString(input_sentence)
                # Evaluate sentence
                output_words = evaluate(encoder, decoder, searcher, voc, input_sentence,emotion,beam_search=beam_search)
                # Format and print response sentence
                output=[]
                for word in output_words:
                    if word == 'PAD':
                        continue
                    elif word == 'EOS':
                        break
                    else:
                        output.append(word)
                print('Bot({}):'.format(emotion_dict[emotion]), ' '.join(output))

        except KeyError:
            print("Error: Encountered unknown word.")
            

# Chat with bot

In [ ]:
# Set dropout layers to eval mode

encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)
searcher2 = BeamSearchDecoder(encoder,decoder,voc.num_words)
# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher2, voc,emo_dict,True)